# WeatherPy

---

## Starter Code to Generate Random Geographic Coordinates and a List of Cities

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Impor the OpenWeatherMap API key
from api_keys import weather_api_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

### Generate the Cities List by Using the `citipy` Library

In [ ]:
# Empty list for holding the latitude and longitude combinations
lat_lngs = []

# Empty list for holding the cities names
cities = []

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(f"Number of cities in the list: {len(cities)}")

---

## Requirement 1: Create Plots to Showcase the Relationship Between Weather Variables and Latitude

### Use the OpenWeatherMap API to retrieve weather data from the cities list generated in the started code

In [ ]:
# Set the API base URL
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"

# Define an empty list to fetch the weather data for each city
city_data = []

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0

    # Create endpoint URL with each city
    params = {
        'units': units,
        'appid': weather_api_key,
        'q': city
    }
    #city_url = # YOUR CODE HERE
    
    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(url, params=params).json()
        # Parse out latitude, longitude, max temp, humidity, cloudiness, wind speed, country, and date
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_humidity = city_weather['main']['humidity']
        city_clouds = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        city_country = city_weather['sys']['country']
        city_date = city_weather['dt']

        # Append the City information into city_data list
        city_data.append({"City": city, 
                          "Lat": city_lat, 
                          "Lng": city_lng, 
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
              
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

In [ ]:
# Convert the cities weather data into a Pandas DataFrame
city_data_df = pd.DataFrame(city_data)

# Show Record Count
city_data_df.count()

In [ ]:
# Display sample data
city_data_df.head()

In [ ]:
# Export the City_Data into a csv
city_data_df.to_csv("output_data/cities.csv", index_label="City_ID")

In [ ]:
# Read saved data
city_data_df = pd.read_csv("output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

### Create the Scatter Plots Requested

#### Latitude Vs. Temperature

In [ ]:
# Build scatter plot for latitude vs. temperature
plt.scatter(city_data_df['Lat'], city_data_df['Max Temp'])

# Incorporate the other graph properties
plt.xlabel('Latitude')
plt.ylabel('Max Temperature (C)')
plt.title(f'City Max Latitude vs. Temperature ({time.strftime("%Y-%m-%d", time.localtime())})')
plt.grid()
# Save the figure
plt.savefig("output_data/Fig1.png")

# Show plot
plt.show()

#### Latitude Vs. Humidity

In [ ]:
# Build the scatter plots for latitude vs. humidity
plt.scatter(city_data_df['Lat'], city_data_df['Humidity'])

# Incorporate the other graph properties
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
plt.title(f'City Max Latitude vs. Humidity ({time.strftime("%Y-%m-%d", time.localtime())})')
plt.grid()

# Save the figure
plt.savefig("output_data/Fig2.png")

# Show plot
plt.show()

#### Latitude Vs. Cloudiness

In [ ]:
# Build the scatter plots for latitude vs. cloudiness
plt.scatter(city_data_df['Lat'], city_data_df['Cloudiness'])

# Incorporate the other graph properties
plt.xlabel('Latitude')
plt.ylabel('Cloudiness (%)')
plt.title(f'City Max Latitude vs. Cloudiness ({time.strftime("%Y-%m-%d", time.localtime())})')
plt.grid()

# Save the figure
plt.savefig("output_data/Fig3.png")

# Show plot
plt.show()

#### Latitude vs. Wind Speed Plot

In [ ]:
# Build the scatter plots for latitude vs. wind speed
plt.scatter(city_data_df['Lat'], city_data_df['Wind Speed'])

# Incorporate the other graph properties
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (m/s)')
plt.title(f'City Max Latitude vs. Wind Speed ({time.strftime("%Y-%m-%d", time.localtime())})')
plt.grid()

# Save the figure
plt.savefig("output_data/Fig4.png")

# Show plot
plt.show()

---

## Requirement 2: Compute Linear Regression for Each Relationship


In [ ]:
# Define a function to create Linear Regression plots
def calc_linear_reg(xvals, yvals):
    (slope, intercept, rvalue, pvalue, stderr) = linregress(xvals, yvals)
    regress_values = xvals * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    return {'line_eq': line_eq, 'rvalue': rvalue, 'regress': regress_values}

In [ ]:
# Create a DataFrame with the Northern Hemisphere data (Latitude >= 0)
northern_hemi_df = city_data_df[city_data_df['Lat'] >= 0]

# Display sample data
northern_hemi_df.head()

In [ ]:
# Create a DataFrame with the Southern Hemisphere data (Latitude < 0)
southern_hemi_df = city_data_df[city_data_df['Lat'] < 0]

# Display sample data
southern_hemi_df.head()

###  Temperature vs. Latitude Linear Regression Plot

In [ ]:
# Linear regression on Northern Hemisphere
xvals = northern_hemi_df['Lat']
yvals = northern_hemi_df['Max Temp']

reg_details = calc_linear_reg(xvals, yvals)

print(f'The r-value is: {reg_details["rvalue"]}')
plt.scatter(xvals,yvals)
plt.plot(xvals,reg_details['regress'],"r-")
plt.annotate(reg_details['line_eq'], (8,-30),fontsize=15,color="red")
plt.xlabel('Latitude')
plt.ylabel('Temperature (C)')
plt.title('Northern Hemisphere: Temp v Latitude')
plt.show()

In [ ]:
# Linear regression on Southern Hemisphere
xvals = southern_hemi_df['Lat']
yvals = southern_hemi_df['Max Temp']

reg_details = calc_linear_reg(xvals, yvals)

print(f'The r-value is: {reg_details["rvalue"]}')
plt.scatter(xvals,yvals)
plt.plot(xvals,reg_details['regress'],"r-")
plt.annotate(reg_details['line_eq'], (-25,11),fontsize=15,color="red")
plt.xlabel('Latitude')
plt.ylabel('Temperature (C)')
plt.title('Southern Hemisphere: Temp v Latitude')
plt.show()

### Discussion about the linear relationship: As expected, the regression equations for the hemispheres confirm that temperature values rise as the lattitude approaches zero (positive slope for the Southern hemisphere). Conversely, as the latitude moves away from the equator (towards +/- 90 degrees) we see a negative slope (or towards cooler temperatures). The northern hemisphere displays a much higher correlation between latitude and temperature - an rscore of -0.85 as opposed to 0.50 for the south.

### Humidity vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
xvals = northern_hemi_df['Lat']
yvals = northern_hemi_df['Humidity']

reg_details = calc_linear_reg(xvals, yvals)

print(f'The r-value is: {reg_details["rvalue"]}')
plt.scatter(xvals,yvals)
plt.plot(xvals,reg_details['regress'],"r-")
plt.annotate(reg_details['line_eq'], (45,20),fontsize=15,color="red")
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
plt.title('Northern Hemisphere: Humidity v Latitude')
plt.show()

In [ ]:
# Southern Hemisphere
xvals = southern_hemi_df['Lat']
yvals = southern_hemi_df['Humidity']

reg_details = calc_linear_reg(xvals, yvals)

print(f'The r-value is: {reg_details["rvalue"]}')
plt.scatter(xvals,yvals)
plt.plot(xvals,reg_details['regress'],"r-")
plt.annotate(reg_details['line_eq'], (-25,11),fontsize=15,color="red")
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
plt.title('Southern Hemisphere: Humidity v Latitude')
plt.show()

### Discussion about the linear relationship: Humidity has a lower correlation to latitude than does the temperature. Here again the northern hemisphere shows greater correlation (0.50 to 0.16). This may partially be due to the higher humidity values of the southern hemisphere - a y-intercept of 77.56 puts more values near the higher limit, and a flatter slope to the regression shows less change across the sample.

### Cloudiness vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
xvals = northern_hemi_df['Lat']
yvals = northern_hemi_df['Cloudiness']

reg_details = calc_linear_reg(xvals, yvals)

print(f'The r-value is: {reg_details["rvalue"]}')
plt.scatter(xvals,yvals)
plt.plot(xvals,reg_details['regress'],"r-")
plt.annotate(reg_details['line_eq'], (48,50),fontsize=15,color="red")
plt.xlabel('Latitude')
plt.ylabel('Cloudiness')
plt.title('Northern Hemisphere: Cloudiness v Latitude')
plt.show()

In [ ]:
# Southern Hemisphere
xvals = southern_hemi_df['Lat']
yvals = southern_hemi_df['Cloudiness']

reg_details = calc_linear_reg(xvals, yvals)

print(f'The r-value is: {reg_details["rvalue"]}')
plt.scatter(xvals,yvals)
plt.plot(xvals,reg_details['regress'],"r-")
plt.annotate(reg_details['line_eq'], (-25,11),fontsize=15,color="red")
plt.xlabel('Latitude')
plt.ylabel('Cloudiness')
plt.title('Southern Hemisphere: Cloudiness v Latitude')
plt.show()

### Discussion about the linear relationship: With regards to the Cloudiness measure, there is a weaker correlation to latitude than in the previous graph sets. Both measure ~ +/- 0.25, registering as fairly weak. What is counterintuitive here is that for Cloudiness, values increase both as you move towards the equator in the Southern Hemisphere, and as you move away from the equator in the Northern hemisphere. This is contradictory to what was seen regarding temperature. 

### Wind Speed vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
xvals = northern_hemi_df['Lat']
yvals = northern_hemi_df['Wind Speed']

reg_details = calc_linear_reg(xvals, yvals)

print(f'The r-value is: {reg_details["rvalue"]}')
plt.scatter(xvals,yvals)
plt.plot(xvals,reg_details['regress'],"r-")
plt.annotate(reg_details['line_eq'], (13,18.5),fontsize=15,color="red")
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (m/s)')
plt.title('Northern Hemisphere: Wind Speed v Latitude')
plt.show()

In [ ]:
# Southern Hemisphere
xvals = southern_hemi_df['Lat']
yvals = southern_hemi_df['Wind Speed']

reg_details = calc_linear_reg(xvals, yvals)

print(f'The r-value is: {reg_details["rvalue"]}')
plt.scatter(xvals,yvals)
plt.plot(xvals,reg_details['regress'],"r-")
plt.annotate(reg_details['line_eq'], (-25,13),fontsize=15,color="red")
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (m/s)')
plt.title('Southern Hemisphere: Wind Speed v Latitude')
plt.show()

### Discussion about the linear relationship: Average windspeed showed very little correlation with latitude, especially in the Northern hemisphere. With an r-score of 0.05 and a regression slope of 0.01, winds would appear to be fairly consistent throughout. The Southern hemisphere shows a slightly higher rate of change (slope = -0.05) as one approaches the equator but does still not appear to be significant. 